In [1]:
import pandas as pd

#Percorsi dei file
file_trip_updates = "C:/Users/C.Marino/Desktop/dataset/trip_updates.txt"
file_risultato_join = "C:/Users/C.Marino/Desktop/dataset/risultato_join_con_stop_times_clean.txt"

trip_updates = pd.read_csv(file_trip_updates, sep=",")  
risultato_join = pd.read_csv(file_risultato_join, sep=",")

#Controllo su trip_id e stop_id
trip_updates['trip_id'] = trip_updates['trip_id'].astype(str)
risultato_join['trip_id'] = risultato_join['trip_id'].astype(str)
trip_updates['stop_id'] = trip_updates['stop_id'].astype(str)
risultato_join['stop_id'] = risultato_join['stop_id'].astype(str)

#Rimuovi le righe con valori mancanti nelle colonne chiave
trip_updates.dropna(subset=['trip_id', 'stop_id'], inplace=True)
risultato_join.dropna(subset=['trip_id', 'stop_id'], inplace=True)

#Controlli
print("Primi record di trip_updates:")
print(trip_updates.head())  #Mostra i primi 5 record di trip_updates

print("\nPrimi record di risultato_join:")
print(risultato_join.head())  #Mostra i primi 5 record di risultato_join


print("\nTipi di dato delle colonne di trip_updates:")
print(trip_updates.dtypes)

print("\nTipi di dato delle colonne di risultato_join:")
print(risultato_join.dtypes)

#Join tra i due DataFrame solo se c'è corrispondenza di 'trip_id' e 'stop_id'
merged_df = pd.merge(trip_updates, risultato_join, on=['trip_id', 'stop_id'], how='inner')

#Verifica della join
if not merged_df.empty:
    print("Join effettuata con successo. Primi record del risultato della join:")
    print(merged_df.head())  

    #Verifica se ci sono righe duplicate nel risultato della join
    if merged_df.duplicated(subset=['trip_id', 'stop_id']).any():
        print("Ci sono righe duplicate nel risultato della join.")

    #Salva il risultato della join in un file chiamato 'real.txt'
    merged_df.to_csv("C:/Users/C.Marino/Desktop/dataset/real.txt", index=False, sep=",")
    print("La join finale è stata salvata in 'real.txt'.")
else:
    print("Nessuna corrispondenza trovata tra trip_updates e risultato_join_con_stop_times_clean.txt.")

real_df = pd.read_csv("C:/Users/C.Marino/Desktop/dataset/real.txt", sep=",")


###Calcolo ritardi

#Controllo tipo di dato della colonna arrival_time_x
print("\nTipo di dato di 'arrival_time_x':")
print(real_df['arrival_time_x'].dtype)


#Controllo tipo di dato della colonna departure_time
print("\nTipo di dato di 'departure_time':")
print(real_df['departure_time'].dtype)


#Converte le colonne 'arrival_time_x' e 'departure_time' in formato datetime
real_df['arrival_time_x'] = pd.to_datetime(real_df['arrival_time_x'], errors='coerce')
real_df['departure_time'] = pd.to_datetime(real_df['departure_time'], errors='coerce')

#Calcola la differenza tra departure_time e arrival_time_x
real_df['delay'] =  real_df['arrival_time_x']  - real_df['departure_time'] 

#Funzione per formattare il ritardo in hh:mm:ss e gestione degli anticipi 
def format_delay(td):
    if pd.isna(td):
        return "00:00:00"
    sign = "-" if td < pd.Timedelta(0) else ""
    abs_td = abs(td)
    hours, remainder = divmod(abs_td.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{sign}{int(hours):02}:{int(minutes):02}:{int(seconds):02}"


real_df['delay_formatted'] = real_df['delay'].apply(format_delay)

#Visualizza i primi 5 record
print("\nPrimi record con la colonna 'delay_formatted':")
print(real_df[['trip_id', 'stop_id', 'arrival_time_x', 'departure_time', 'delay_formatted']].head())

#Salva il risultato con la nuova colonna
real_df.to_csv("C:/Users/C.Marino/Desktop/dataset/real_with_delay_and_early.txt", index=False, sep=",")
print("\nIl risultato con ritardi e anticipi è stato salvato in 'real_with_delay_and_early.txt'.")

###Filtro sui ritardi superiori a 5minuti

ritardi_consistenti = real_df[real_df['delay'] > pd.Timedelta(minutes=5)]

print("\nTrip_ID e Stop_ID con ritardi superiori a 5 minuti:")
print(ritardi_consistenti[['trip_id', 'stop_id', 'delay_formatted']].head())

#Risultato salvato su file 'ritardi_consistenti.txt'
ritardi_consistenti.to_csv("C:/Users/C.Marino/Desktop/dataset/ritardi_consistenti.txt", index=False, sep=",")
print("\nI ritardi superiori a 5 minuti sono stati salvati in 'ritardi_consistenti.txt'.")



C:\Users\C.Marino\AppData\Local\Temp\ipykernel_21328\3748727649.py:8: DtypeWarning: Columns (0,2,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  risultato_join = pd.read_csv(file_risultato_join, sep=",")


Primi record di trip_updates:
   trip_id stop_id arrival_date arrival_time  arrival_time_utc
0  0#611-4   71671   2025-02-10     08:55:46               NaN
1  0#611-4   71672   2025-02-10     08:57:12               NaN
2  0#611-4   71673   2025-02-10     08:58:14               NaN
3  0#611-4   71674   2025-02-10     08:59:24               NaN
4  0#611-4   71675   2025-02-10     09:02:48               NaN

Primi record di risultato_join:
  route_id agency_id route_short_name  route_long_name  route_type  \
0      211       OP1              211              NaN           3   
1      211       OP1              211              NaN           3   
2      211       OP1              211              NaN           3   
3      211       OP1              211              NaN           3   
4      211       OP1              211              NaN           3   

                                      route_url  route_color  \
0  https://muoversiaroma.it/it/linea?numero=211          NaN   
1  https:/

C:\Users\C.Marino\AppData\Local\Temp\ipykernel_21328\3748727649.py:68: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  real_df['arrival_time_x'] = pd.to_datetime(real_df['arrival_time_x'], errors='coerce')
C:\Users\C.Marino\AppData\Local\Temp\ipykernel_21328\3748727649.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  real_df['departure_time'] = pd.to_datetime(real_df['departure_time'], errors='coerce')



Primi record con la colonna 'delay_formatted':
   trip_id  stop_id      arrival_time_x      departure_time delay_formatted
0  0#611-4    71671 2025-03-25 08:55:46 2025-03-25 08:31:54        00:23:52
1  0#611-4    71671 2025-03-25 09:06:46 2025-03-25 08:31:54        00:34:52
2  0#611-4    71671 2025-03-25 09:12:50 2025-03-25 08:31:54        00:40:56
3  0#611-4    71671 2025-03-25 09:03:56 2025-03-25 08:31:54        00:32:02
4  0#611-4    71672 2025-03-25 08:57:12 2025-03-25 08:33:04        00:24:08

Il risultato con ritardi e anticipi è stato salvato in 'real_with_delay_and_early.txt'.

Trip_ID e Stop_ID con ritardi superiori a 5 minuti:
   trip_id  stop_id delay_formatted
0  0#611-4    71671        00:23:52
1  0#611-4    71671        00:34:52
2  0#611-4    71671        00:40:56
3  0#611-4    71671        00:32:02
4  0#611-4    71672        00:24:08

I ritardi superiori a 5 minuti sono stati salvati in 'ritardi_consistenti.txt'.
